In [ ]:
!pip install keras-rl2

In [2]:
import gym
import tensorflow as tf

In [3]:
from rl.agents.dqn import DQNAgent

In [4]:
env_name = 'CartPole-v0'
env = gym.make(env_name)

In [5]:
env.reset()
for step in range(200):
    env.render(mode="human")
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
env.close()

d:\Anaconda\lib\site-packages\gym\logger.py:34: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [5]:
n_actions = env.action_space.n
n_obs = env.observation_space.shape

model   = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(1,) + n_obs),  
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(n_actions, activation='linear')
])

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['mse'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 2)                 66        
                                                                 
Total params: 690
Trainable params: 690
Non-trainable params: 0
_________________________________________________________________


d:\Anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [7]:
from rl.memory import SequentialMemory
memory = SequentialMemory(limit=20000, window_length=1)

from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=20000)

In [8]:
dqn = DQNAgent(model =model, nb_actions = n_actions, memory = memory, nb_steps_warmup=10, target_model_update=100, policy=policy)  

In [9]:
dqn.compile(tf.keras.optimizers.Adam(lr=0.001), metrics=['mae'])

d:\Anaconda\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

In [11]:
dqn.save_weights('dqn_{}_weights.h5f'.format(env_name), overwrite=True)

In [13]:
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
